# Gomory Chvátat cuts

### Given LP

min ${ c^Tx : Ax =< b , x \in Z_+^{n}} , A \in Z^{m x n} , b \in Z^m , c \in R^n $

the inequality

$ \lfloor \lambda^TA \rfloor x \leq \lfloor \lambda^Tb \rfloor , \lambda \in R_{\geq 0}^m$

1. Formulate model
2. Sample with $\lambda \in R_+^m$
3. Make GC-cut
4. Add inequality to model

In [18]:
using JuMP
using SDPA
using SCIP

# KROK 1

In [19]:
model = Model(SCIP.Optimizer)

A JuMP Model
├ solver: SCIP
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [20]:
n = 2
c = [1 ; 1]
A = [4 1; 1 4; 1 -1;]
b = [28; 27; 1]

3-element Vector{Int64}:
 28
 27
  1

In [21]:
@variable(model, x[1:n] >= 0, Int)

2-element Vector{VariableRef}:
 x[1]
 x[2]

In [22]:
@objective(model, Max, c' * x)

x[1] + x[2]

In [23]:
@constraint(model, A * x <= b )

[4 x[1] + x[2] - 28, x[1] + 4 x[2] - 27, x[1] - x[2] - 1] in Nonpositives()

In [24]:
optimize!(model)

feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+000
presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 3 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 0 del conss, 0 add conss, 4 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver found nothing
(round 3, exhaustive) 0 del vars, 0 del conss, 0 add conss, 4 chg bounds, 0 chg sides, 0 chg coeffs, 3 upgd conss, 0 impls, 0 clqs
(round 4, medium)     0 del vars, 0 del conss, 0 add conss, 5 chg bounds, 0 chg sides, 0 chg coeffs, 3 upgd conss, 0 impls, 0 clqs
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (5 rounds: 5 fast, 3 medium, 2 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 5 tightened bounds, 0 added

# KROK 2

In [25]:
# lambda = rand(0:10, 3) # vector Z

lambda = rand([i / 10 for i in range(0, 101)], 3) # vector R

3-element Vector{Float64}:
 5.2
 5.8
 1.5

In [26]:
x_result = value.(x)

2-element Vector{Float64}:
 5.0
 5.0

In [27]:
print((lambda' * A) * x)
left = floor.(lambda' * A) * x

28.1 x[1] + 26.9 x[2]

1-element Vector{AffExpr}:
 28 x[1] + 26 x[2]

In [28]:
print(lambda' * b)
right = floor.(lambda' * b)

303.7

303.0

In [29]:
# left <= right

# KROK 3 I 4

In [30]:
@constraint(model, left .<= right)

1-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 28 x[1] + 26 x[2] <= 303

In [31]:
optimize!(model)

4/4 feasible solutions given by solution candidate storage, new primal bound 1.000000e+001

presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 3 chg bounds, 1 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 2 del conss, 0 add conss, 7 chg bounds, 2 chg sides, 1 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (3 rounds: 3 fast, 0 medium, 0 exhaustive):
 4 deleted vars, 2 deleted constraints, 0 added constraints, 9 tightened bounds, 0 added holes, 2 changed sides, 1 changed coefficients
 0 implications, 0 cliques
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.00
Solving Nodes      : 0
Primal Bound       : +1.00000000000000e+001 (4 solutions)
Dual Bound         : +1.00000000000000e+001
Gap                : 0.00 %


In [32]:
value.(x)

2-element Vector{Float64}:
 5.0
 5.0

# KROK 5 - powtarzanie

In [33]:
set_silent(model)
for j in 1:1000
    lambda = rand([i / 10 for i in range(0, 101)], 3)
    left = floor.(lambda' * A) * x
    right = floor.(lambda' * b)
    @constraint(model, left .<= right)
    optimize!(model)

    if (j%100==0)
        println(value.(x))
        println(objective_value(model))
        println("-------------")
    end
end

[5.0, 5.0]
10.0
-------------
[5.0, 5.0]
10.0
-------------
[5.0, 5.0]
10.0
-------------
[5.0, 5.0]
10.0
-------------
[5.0, 5.0]
10.0
-------------
[5.0, 5.0]
10.0
-------------
[5.0, 5.0]
10.0
-------------
[5.0, 5.0]
10.0
-------------
[5.0, 5.0]
10.0
-------------
[5.0, 5.0]
10.0
-------------
